In [ ]:
import ee
import geemap
from gee_0_utils import *
from gee_1_age_agbd import desired_lulc

initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder

age, lulc, fire = desired_lulc()
mask_1985 = False
history_interval = 5
aggregate_LU_types = True

## Land Use Land Cover

### Grouping land use types (if aggregation is desired)

- annual_crops = 39, 20, 40, 62, 41
- perennial_crops = 46, 47, 35, 48
- pasture = 15
- mosaic = 21
- forest plantation = 9


In [ ]:

# Listing land use types that are NOT forest
if (aggregate_LU_types):
    
    LU_index = [10,20,30,40,50]

    # A flat list of pixel values to replace.
    from_list = [15, 46, 47, 35, 48, 39, 20, 40, 62, 41, 21, 9, 3]

    # A corresponding list of new values.
    to_list = [10, 20, 20, 20, 20, 30, 30, 30, 30, 30, 40, 50, 3]

    # for each forest age, mask the lulc of the year immediately preceding abandonment
    lulc_aggregated = ee.Image()
    for band in lulc.bandNames().getInfo():
        lu_yr = lulc.select(band)
        remapped_band = lu_yr.remap(from_list, to_list).rename(band)
        lulc_aggregated = lulc_aggregated.addBands(remapped_band)

    lulc = lulc_aggregated.slice(1).byte()
else:
    LU_index = [15, 39, 20, 40, 62, 41, 46, 47, 35, 48, 9, 21]



### Restricting history to n years before abandonment

Since we only have land use history dating back to 1985, the data is censored. In order to restrict history to exclude pixels with possible long histories of anthropogenic use pre-1985 which could not be included in the analysis:

- Make a mask with regrowing pixels that were forested in 1985 (keep only the patches that were forested in 1985, then subsequently deforested, and regrew after that).

- Making a mask including only pixels that have a specified number of bands (hist_interval) between the first and the last occurrence of an anthropogenic pixel (desired extent of land use history to include)

In [ ]:

# for each year, make a mask for all anthropogenic pixels, and multiply by the year to have the
# pixel value be the year of observation of that land use type.
non_forest_years = ee.Image()
for yr in range(1985, 2020):
    year = f"{yr}"
    lu_yr = lulc.select(year).neq(3).multiply(yr)
    non_forest_years = non_forest_years.addBands(lu_yr)

# find the minimum and maximum non-zero value per pixel across all bands
# (first and last year of anthropogenic land use)
first_anthro = non_forest_years.selfMask().reduce(ee.Reducer.min())

if mask_1985:
    last_anthro = non_forest_years.reduce(ee.Reducer.max())
    # get total of continuous years from the first and last observation of anthropogenic land use
    anthro_gaps = last_anthro.subtract(first_anthro).add(1)

    # select pixels that were forested in 1985
    # and that have at least history_interval years of anthropogenic land use
    lulc = lulc.updateMask(lulc.select("1985").eq(3)).selfMask() \
    .updateMask(age.lt(35-history_interval)).updateMask(anthro_gaps.lte(history_interval))


### Total sum of years under each desired land use type

For each desired land use type, the total occurrences per pixel are summed

In [ ]:
LU_sum = ee.Image()
for val in LU_index:
    lulc_val = lulc.eq(val)
    num_cells = lulc_val.reduce(ee.Reducer.sum()).rename(f"lulc_sum_{val}")
    LU_sum = LU_sum.addBands(num_cells)

# removes the first (empty) band
LU_sum = LU_sum.slice(1).byte()

### Last Land Use Type

Finds what was the last land use type observed before abandonment.

In [ ]:
# for each forest age, mask the lulc of the year immediately preceding abandonment
last_LU = ee.Image()
for yr in config.range_1986_2019:
    year = f"{yr}"
    lu_yr = lulc.select(year)
    age_mask = age.eq(config.config.last_year - yr)
    last_LU = last_LU.addBands(lu_yr.updateMask(age_mask))

# remove first empty band
# merge all images by adding them up
last_LU = last_LU.slice(1).reduce(ee.Reducer.sum()).rename("last_LU").byte()

### Fallow period length

Finds the total number of years that land was left fallow (classified as forest before regrowth)

In [ ]:
nat_cover = lulc.eq(3)
total_nat_cover = nat_cover.reduce(ee.Reducer.sum())
mat_for_years = anthro_gaps.subtract(history_interval).multiply(-1).rename("mat_for_years").byte()
fallow = total_nat_cover.subtract(36).subtract(mat_for_years).add(10).rename("fallow").byte()

nat_cover = fallow.addBands(mat_for_years)

## Fire

### Total number of fires per pixel

Each burned pixel contains the value of the land use type that burned (such as the desired_values vector specified in the land use section) - unburned pixels are zero.

There are instances of fires:
- after the first observation of anthropogenic land use
- before the first observation of anthropogenic land use (forest fire)
- after the last observation of anthropogenic land use (burn in regrowing forest)

*Note that fire has different transform than lulc, and the projections will be adjusted when exporting.*

### How many years ago was the last fire?

Make one image with 34 bands, each with the number of years from the instance of burn to 2020.
Each pixel is 0 if unburned, and "time since fire" in number of years if it's burned.

In [ ]:
# Making a fire mask with pixels of value 1 for burned pixels and 0 otherwise instead:
fire = fire.gt(0)

"""
Establishing a specific range for land use history for analysis, the ages of forests regrowing are also restricted.
For example, if we have a fixed 10 years of history:
1985 ---------- 10 years ----------- 1995 ----------- 25 years -----------2020
the regrowing forests have started regrowing in 1995 at the very earliest.

Then, the years of regrowth considered are:
"""

if mask_1985 == True:
    first_year_anthro = config.first_year + 1 + history_interval
    years_regrowth_considered = range((first_year_anthro), config.last_year)
else:
    years_regrowth_considered = years

# Initialize images
num_fires_before_regrowth = ee.Image()
num_fires_after_regrowth = ee.Image()

ts_fire_before_regrowth = ee.Image()
ts_fire_after_regrowth = ee.Image()

num_fires_before_first_anthro = ee.Image()
num_fires_during_anthro = ee.Image()

ts_all_fires = ee.Image()

"""
for each year, multiply the fire mask by the year to have the pixel value be the year of observation of each fire
"""
for yr in range(1985, 2020):
    ts_all_fires = ts_all_fires.addBands(
        fire.select([f"{yr}"]).multiply(config.last_year - yr)
    )


def add_fire_bands(image, start_year, end_year, mask, input):
    """
    Adds bands to an image for a range of years, applying a mask and a reducer.
    """

    bands = input.select(
        [f"{year}" for year in range(start_year, end_year)]
    ).updateMask(mask)

    if input == ts_all_fires:
        return image.addBands(bands.reduce(ee.Reducer.lastNonNull()).byte())
    else:
        return image.addBands(bands.reduce(ee.Reducer.sum()).byte())


"""
Calculate burn information and add yearly bands to the corresponding images
"""
for yr in years_regrowth_considered:
    yrs_ago = config.last_year - yr
    age_mask = age.eq(yrs_ago)

    # Add bands for after regrowth
    num_fires_after_regrowth = add_fire_bands(
        num_fires_after_regrowth, yr, config.last_year, age_mask, fire
    )
    # Add bands for before regrowth
    num_fires_before_regrowth = add_fire_bands(
        num_fires_before_regrowth, config.first_year, yr, age_mask, fire
    )

    # Add bands for after regrowth
    ts_fire_after_regrowth = add_fire_bands(
        ts_fire_after_regrowth, yr, config.last_year, age_mask, ts_all_fires
    )
    # Add bands for before regrowth
    ts_fire_before_regrowth = add_fire_bands(
        ts_fire_before_regrowth, config.first_year, yr, age_mask, ts_all_fires
    )

    if mask_1985:
        # Add bands for before first anthro
        num_fires_before_first_anthro = add_fire_bands(
            num_fires_before_first_anthro,
            config.first_year,
            yr - history_interval,
            age_mask,
            fire,
        )
        # Add bands for during anthro
        num_fires_during_anthro = add_fire_bands(
            num_fires_during_anthro, yr - history_interval, yr, age_mask, fire
        )

def process_band(band, name):
    """
    For all images, we have one band per year. Across these bands, there is only one value per pixel.
    To get all the pixels into a single layer, we can sum the values across bands.
    """
    return band.slice(1).reduce(ee.Reducer.sum()).rename(name)


# Apply the helper function to each band
fires = (
    process_band(num_fires_after_regrowth, "num_fires_after_regrowth")
    .addBands(
        [
            process_band(num_fires_before_regrowth, "num_fires_before_regrowth"),
            process_band(ts_fire_before_regrowth, "ts_fire_before_regrowth"),
            process_band(ts_fire_after_regrowth, "ts_fire_after_regrowth"),
        ]
    )
    .byte()
)

# Slice and reduce the final image
if mask_1985:
    fires = fires.addBands(
        [
            process_band(num_fires_during_anthro, "num_fires_during_anthro"),
            process_band(
                num_fires_before_first_anthro, "num_fires_before_first_anthro"
            ),
        ]
    ).byte()


fires = fires.unmask(0)

### Export land_use

In [ ]:
land_use = LU_sum.addBands([last_LU, nat_cover, fires])

name = ""
if aggregate_LU_types:
    name += "_aggregated"
if mask_1985:
    name += f"_{history_interval}_yr"

export_image(land_use, f"land_use{name}", scale = 30)